In [1]:
# this notebook is for the new image which comes , so this image will be able to generate caption for that new image 

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import json 
import pickle
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from tensorflow.keras.layers import add

In [2]:

model = load_model("model_weights/model_9.h5")
#model._make_predict_function()

In [3]:
model_temp = ResNet50(weights="imagenet", input_shape=(224,224,3))


In [4]:

# Create a new model, by removing the last layer (output layer of 1000 classes) from the resnet50
model_resnet = Model(model_temp.input, model_temp.layers[-2].output)
#model_resnet._make_predict_function()

In [5]:
def preprocess_image(img):
    img = image.load_img(img, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


In [6]:

def encode_image(img):
    img = preprocess_image(img)
    feature_vector = model_resnet.predict(img)
    feature_vector = feature_vector.reshape(1, feature_vector.shape[1])
    return feature_vector

In [7]:
enc = encode_image("images.jpeg")

1/1 [==============================] - 2s 2s/step


In [8]:
enc.shape

(1, 2048)

In [9]:

with open("./storage/word_to_idx.pkl", 'rb') as w2i:
    word_to_idx = pickle.load(w2i)
    
with open("./storage/idx_to_word.pkl", 'rb') as i2w:
    idx_to_word = pickle.load(i2w)


In [10]:

def predict_caption(photo):
    in_text = "startseq"
    max_len = 35 #maxlen  
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence], maxlen=max_len, padding='post')

        ypred =  model.predict([photo,sequence])
        ypred = ypred.argmax()
        word = idx_to_word[ypred]
        in_text+= ' ' +word
        
        if word =='endseq':
            break
        
        
    final_caption =  in_text.split()
    final_caption = final_caption[1:-1]
    final_caption = ' '.join(final_caption)
    
    return final_caption


In [11]:
enc = encode_image("images.jpeg")

1/1 [==============================] - 0s 169ms/step


In [12]:
predict_caption(enc)

1/1 [==============================] - 0s 60ms/step


'dog is running through the grass'

In [ ]:
def caption_this_image(image):

    enc = encode_image(image)
    caption = predict_caption(enc)
    return caption